# Testing the pairwise ANNs for face classification

In [1]:
from numpy import load
import numpy as np

y = load('yaleExtB_target.npy')
X = load('yaleExtB_data.npy')

In [2]:
X[:5]

array([[ 82.,  86.,  86., ...,  91.,  85.,  80.],
       [ 96.,  99., 107., ...,  68.,  62.,  57.],
       [ 98., 104., 117., ...,  35.,  34.,  33.],
       [ 21.,  20.,  20., ...,  21.,  21.,  22.],
       [ 64.,  62.,  61., ..., 114., 112., 106.]])

In [3]:
print('Data shape', X.shape)

Data shape (1500, 5236)


In [4]:
y[:-5]

array([ 2.,  2.,  2., ..., 39., 39., 39.])

In [5]:
from sklearn.model_selection import train_test_split

# split data for training and testing
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size=0.33,
                                                    shuffle=True)

In [6]:
from sklearn.decomposition import PCA

# PCA 
nof_prin_components = 200  # PARAMETER for optimisation in expereiments
pca = PCA(n_components=nof_prin_components, whiten=True).fit(X_train)
# applies PCA to the train and test images to calculate the principal components
X_train_pca = pca.transform(X_train) 
X_test_pca = pca.transform(X_test)

In [7]:
# Pairwise classifier (one vs one classifier)

from sklearn.multiclass import OneVsOneClassifier
from sklearn.svm import LinearSVC

clf = OneVsOneClassifier(LinearSVC())

In [8]:
# train
model = clf.fit(X_train_pca, y_train)

In [9]:
# Use the model/predictor to predict
y_pred = model.predict(X_test_pca)

In [12]:
from sklearn import metrics

# view the accuracy
print("Accuracy:", metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.9676767676767677


## A name predictor for face

In [13]:
target_names = {'2':'Johnathan',
'3':'Anthony',
'4':'Victor',
'5':'Joshua',
'6':'Albert',
'7':'Michael',
'8':'Anthony',
'9':'Brian',
'11':'William',
'12':'Patrick',
'13':'Adrian',
'15':'Jared',
'16':'Christopher',
'17':'Geoffrey',
'18':'Bryan',
'20':'Gregory',
'22':'Dustin',
'23':'Devon',
'24':'Matthew',
'25':'Tyler',
'26':'John',
'27':'Brian',
'28':'Michael',
'32':'Tommy',
'33':'Christopher',
'34':'Miguel',
'35':'Alexander',
'37':'Jeffery',
'38':'Steven',
'39':'Tom',
               }

In [14]:
def get_name(class_index):
    if str(class_index) in target_names.keys():
        return target_names[str(class_index)]
    else:
        return 'Unidentified'

In [16]:
import cv2

from itertools import chain
# load a random image
test_image = cv2.imread("./Tr0/yaleB11_P00A-015E+20.jpg")

gray = cv2.cvtColor(test_image, cv2.COLOR_BGR2GRAY)

# flatten the image data
image = [j for i in gray for j in i]

# get PCA of the test image
test_pca = pca.transform([image])

# predict the image class
class_pred = model.predict(test_pca)

print(class_pred)

print(get_name(int(class_pred)))

[11.]
William


## Live with webcam

In [17]:
faceCascade = cv2.CascadeClassifier('haarcascades/haarcascade_frontalface_alt2.xml')

In [18]:
video_capture = cv2.VideoCapture(0)

# for text display in cv2
font = cv2.FONT_HERSHEY_SIMPLEX
fontScale = 1   
color = (255, 0, 0)  
thickness = 2

while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

   
    faces = faceCascade.detectMultiScale(
        gray,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(30, 30),
        flags=cv2.CASCADE_SCALE_IMAGE
    )

    # Draw a rectangle around the faces
    # extract the face
    # flatten the face data
    # calculate pca
    # classify the face then write name
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        face = frame[y:y+h, x:x+w]   
        resize_face = cv2.resize(face, (68,77))        
        gray_face = cv2.cvtColor(resize_face, cv2.COLOR_BGR2GRAY)
        flat_face = [j for i in gray_face for j in i]       
        test_pca = pca.transform([flat_face])        
        class_pred = model.predict(test_pca)
        name = get_name(int(class_pred))
        cv2.putText(frame, name, (x, y-10), font, fontScale, color, thickness, cv2.LINE_AA)     

    # Display the resulting frame
    cv2.imshow('Video', frame)
   
    # cv2.imshow('Video', gray)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything is done, release the capture
video_capture.release()
cv2.destroyAllWindows()